In [1]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from glob import glob

import os
import pandas as pd
import numpy as np
import cv2

In [2]:
ensembling_models = {
    'DenseNet201-Fine-Tune': {'img_size': 299, 'mean_proc': 'DenseNet'},
    'Xception-Fine-Tune': {'img_size': 299,  'mean_proc': None},
    'InceptionResNetV2': {'img_size': 229,  'mean_proc': None}
}

In [3]:
img_size = 299

In [4]:
def load_test_data(Gray2RGB=False, mean_proc=False, img_size=img_size):
    img_path_list = []
    #找出 rest data 的路徑
    for img_path in glob('{}/*.png'.format(testset_dir)):
        img_path_list.append(img_path)
        
    X_test = []
    X_id = []
    for path in img_path_list:
        img = cv2.imread(path, 0)
        img = cv2.resize(img, (img_size, img_size))
        
        if Gray2RGB == True:
            img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img = img.astype(np.float32)

        if mean_proc == 'VGG16_ImageNet':
            img = img - np.array([123.68, 116.779, 103.939])
            img = img[:,:,::-1]  # RGB to BGR
            img = (img - np.min(img)) / np.max(img)
        if mean_proc == 'DenseNet':
            #img /= 255.
            img /= 299
            mean = [0.485, 0.456, 0.406]
            std = [0.229, 0.224, 0.225]
            img = (img - mean) / std
        else:
            #img /= 255.
            img /= 299
            img -= 0.5
            img *= 2.
            
        img_id = path.split('/')[-1].split('.')[0]
        X_test.append(img)
        X_id.append(img_id)
        
    X_test = np.array(X_test, dtype=np.float32)
    
    if Gray2RGB == False:
        X_test = np.reshape(X_test, X_test.shape+(1,))
    
    return X_test, X_id

In [5]:
# 指定存放 test 資料集的資料夾
testset_dir ="./testing_set"
testset_list = []
test_id_list = []

for img_path in glob('{}/*.png'.format(testset_dir)):
    testset_list.append(img_path)
    id = img_path.split('/')[-1].split('.')[0]  # -1 表示圖片路徑的最後一個 part，也就是圖片名，由於圖片名有加副檔名，
    test_id_list.append(id)                     # 所以用 '.' 做切割，取第0個
testset_df = pd.DataFrame({'id': test_id_list, 'path': testset_list}).sort_values(by='id')

In [6]:
for key in ensembling_models.keys():
    print(key)
    img_size = ensembling_models[key]['img_size']
    mean_proc = ensembling_models[key]['mean_proc']
    model = load_model('./saved_models/{}.h5'.format(key))
    X_test, X_id = load_test_data(Gray2RGB=True, mean_proc=mean_proc, img_size=img_size)
    ensembling_models[key]['y_test_pred_prob'] = model.predict(X_test)

DenseNet201-Fine-Tune
Xception-Fine-Tune


In [7]:
num_models = float(len(ensembling_models.keys()))
y_test_pred_prob = 0
for key in ensembling_models.keys():
    y_test_pred_prob += ensembling_models[key]['y_test_pred_prob']

y_test_pred_prob /= num_models

In [8]:
y_test_pred = y_test_pred_prob.argmax(axis=-1)

# load_test_data

In [9]:
import numpy as np
import pandas as pd
#X_test, X_id = load_test_data(Gray2RGB=True, mean_proc='VGG16_ImageNet', img_size=img_size)
X_test, X_id = load_test_data(Gray2RGB=True, mean_proc=None, img_size=img_size)
y_test_pred_df = pd.DataFrame({'id': np.array(X_id), 'class':y_test_pred}).sort_values(by='id')

In [10]:
y_test_pred_df.to_csv('./ensemble.csv', index=False)